<a href="https://colab.research.google.com/github/ManciniGa/Aprendizado-Profundo/blob/main/OCR_Detec%C3%A7%C3%A3o_de_texto_com_EAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de texto em imagens com EAST detector

- Paper da técnica: https://arxiv.org/pdf/1704.03155v2.pdf


Este código implementa um processo de detecção de texto em imagens utilizando o Tesseract OCR, com foco na extração de texto de regiões específicas da imagem. Primeiramente, ele detecta as caixas delimitadoras ao redor das regiões de texto e, em seguida, ajusta essas caixas para a escala original da imagem. Após isso, ele extraí a região de interesse (ROI) de cada caixa e aplica uma margem adicional para capturar mais contexto ao redor do texto detectado. A imagem com as caixas e margens é exibida, e o Tesseract OCR é utilizado para realizar o reconhecimento de texto em cada ROI extraída. O texto extraído é impresso no console, permitindo validar o processo de OCR. A visualização das caixas e o texto extraído são essenciais para revisar e ajustar as detecções realizadas.

# Importando as bibliotecas

In [ ]:
# Importa a biblioteca OpenCV, utilizada para processamento de imagens e visão computacional.
import cv2

# Importa a biblioteca NumPy, que facilita manipulações de arrays e cálculos matemáticos.
import numpy as np

# Importa a função cv2_imshow do módulo google.colab.patches.
# Essa função é usada para exibir imagens no Google Colab, já que cv2.imshow não funciona em navegadores.
from google.colab.patches import cv2_imshow

# Importa a função non_max_suppression do módulo imutils.object_detection.
# Essa função é usada para reduzir sobreposição de caixas delimitadoras em detecção de objetos,
# mantendo apenas as mais relevantes.
from imutils.object_detection import non_max_suppression


# Conectando ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copia o arquivo `frozen_east_text_detection.pb` do Google Drive para o diretório atual.
# O arquivo está localizado no caminho especificado dentro do Google Drive.
!cp /content/drive/MyDrive/Cursos\ -\ recursos/OCR\ com\ Python/Modelos/frozen_east_text_detection.pb ./


In [ ]:
# Copia recursivamente o diretório "Imagens" localizado no Google Drive
# para o diretório "images/" no ambiente atual.
# O caminho de origem inclui caracteres especiais, como espaços, que são escapados com `\`.
!cp -R /content/drive/MyDrive/Cursos\ -\ recursos/OCR\ com\ Python/Imagens images/


# Processando a imagem

In [ ]:
# Define o caminho para o modelo de detecção de texto treinado.
# O modelo "frozen_east_text_detection.pb" é um modelo baseado na arquitetura EAST (Efficient and Accurate Scene Text Detection).
detector = 'frozen_east_text_detection.pb'

# Define as dimensões de entrada para a rede neural.
# largura: 320 pixels
# altura: 320 pixels
# Essas dimensões são geralmente usadas para redimensionar as imagens antes de serem processadas pelo modelo.
largura, altura = 320, 320


In [ ]:
# Define o caminho para a imagem que será utilizada para a detecção de texto.
# Neste caso, a imagem é "diploma.jpg", localizada dentro do diretório "images/".
imagem = 'images/diploma.jpg'

# Define o valor mínimo de confiança para considerar uma detecção válida.
# Apenas regiões com uma confiança maior ou igual a 0.9 serão mantidas.
# O valor pode ser ajustado no intervalo de 0.0 a 1.0, onde valores mais altos tornam o modelo mais restritivo.
min_confianca = 0.9  # @param {type: 'number', min:0.0, max:1.0}


In [ ]:
# Lê a imagem do caminho especificado usando a função cv2.imread.
# A imagem é carregada como um array NumPy, onde cada pixel é representado por valores de cor (BGR).
img = cv2.imread(imagem)

# Exibe a imagem carregada no ambiente do Google Colab.
# A função cv2_imshow é usada em vez de cv2.imshow porque cv2.imshow não funciona em navegadores.
cv2_imshow(img)


NameError: name 'cv2' is not defined

In [ ]:
# Cria uma cópia da imagem original.
# Isso é feito para preservar a imagem original enquanto se aplica processamento na cópia.
original = img.copy()


In [ ]:
# Obtém as dimensões da imagem carregada.
# A função `.shape` retorna uma tupla com as dimensões (altura, largura, número de canais de cor).
img.shape

(480, 750, 3)

In [ ]:
# Obtém a altura da imagem a partir da primeira dimensão do array (número de linhas).
H = img.shape[0]

# Obtém a largura da imagem a partir da segunda dimensão do array (número de colunas).
W = img.shape[1]

# Imprime os valores de altura (H) e largura (W) da imagem.
print(H, W)

480 750


In [ ]:
# Calcula a proporção de redimensionamento para a largura.
# Isso é feito dividindo a largura original da imagem (W) pela largura-alvo especificada (variável `largura`).
proporcao_W = W / float(largura)

# Calcula a proporção de redimensionamento para a altura.
# Isso é feito dividindo a altura original da imagem (H) pela altura-alvo especificada (variável `altura`).
proporcao_H = H / float(altura)

# Imprime as proporções de redimensionamento para a largura e a altura.
print(proporcao_W, proporcao_H)


2.34375 1.5


In [ ]:
# Redimensiona a imagem para as dimensões especificadas (largura, altura).
# A função cv2.resize ajusta a imagem original para corresponder às dimensões desejadas.
# Aqui, a imagem será redimensionada para 320x320 pixels, conforme definido anteriormente.
img = cv2.resize(img, (largura, altura))

# Após o redimensionamento, atualiza o valor da altura da imagem redimensionada.
# A nova altura será igual ao valor especificado na variável `altura`.
H = img.shape[0]

# Após o redimensionamento, atualiza o valor da largura da imagem redimensionada.
# A nova largura será igual ao valor especificado na variável `largura`.
W = img.shape[1]

# Imprime as novas dimensões da imagem (altura e largura) para confirmar o redimensionamento.
print(H, W)

320 320


In [ ]:
# Exibe a imagem redimensionada no ambiente do Google Colab.
# Após o redimensionamento, a função `cv2_imshow` mostra a imagem para que possamos verificar o resultado.
cv2_imshow(img)

NameError: name 'cv2_imshow' is not defined

# Carregando a rede neural

In [ ]:
# Define os nomes das camadas de saída do modelo EAST para detecção de texto.
# Essas camadas serão utilizadas para obter os resultados do modelo após o processamento da imagem.

# 'feature_fusion/Conv_7/Sigmoid':
#   Essa camada é responsável por fornecer o mapa de probabilidades de texto.
#   Indica a probabilidade de cada pixel pertencer a uma região de texto.

# 'feature_fusion/concat_3':
#   Essa camada fornece as coordenadas geométricas das caixas delimitadoras (bounding boxes)
#   para cada região identificada como texto.
nomes_camadas = ['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3']

In [ ]:
# Carrega o modelo de rede neural pré-treinado para detecção de texto.
# O modelo especificado pelo caminho armazenado na variável 'detector' (frozen_east_text_detection.pb)
# é carregado usando a função `cv2.dnn.readNet`, que faz parte do módulo de Redes Neurais Profundas do OpenCV.
rede_neural = cv2.dnn.readNet(detector)

In [ ]:
# Exibe as dimensões da imagem atualmente armazenada na variável `img`.
# A função `.shape` retorna uma tupla no formato (altura, largura, canais),
# que descreve a estrutura do array que representa a imagem.
img.shape


(320, 320, 3)

In [ ]:
# Prepara a imagem para ser usada como entrada na rede neural.
# A função `cv2.dnn.blobFromImage` converte a imagem em um "blob", que é o formato necessário para as redes neurais do OpenCV.
blob = cv2.dnn.blobFromImage(
    img,           # Imagem de entrada que será processada.
    1.0,           # Fator de escala para normalizar os valores dos pixels (neste caso, sem escala adicional).
    (W, H),        # Dimensões da imagem de entrada que a rede espera (largura e altura da imagem redimensionada).
    swapRB=True,   # Indica que os canais devem ser convertidos de BGR para RGB.
    crop=False     # Especifica que a imagem não deve ser cortada durante o redimensionamento.
)


In [ ]:
blob.shape # batch_size

(1, 3, 320, 320)

In [ ]:
# Configura o blob gerado como entrada para a rede neural.
# O método `setInput` atribui o blob (que contém a imagem processada) como entrada para o modelo carregado.
rede_neural.setInput(blob)

# Realiza a inferência na imagem para obter as saídas das camadas especificadas.
# A função `forward` executa a passagem direta do blob pela rede neural, retornando os resultados das camadas definidas.
# Aqui, os resultados das camadas 'feature_fusion/Conv_7/Sigmoid' (scores) e 'feature_fusion/concat_3' (geometry) são extraídos.
scores, geometry = rede_neural.forward(nomes_camadas)


In [ ]:
# Exibe o conteúdo da variável `scores`, que contém o mapa de confiança.
# `scores` é uma saída da rede neural e representa a probabilidade de cada região na imagem ser texto.
# Geralmente, é um tensor de 4 dimensões no formato (N, C, H, W), onde:
# - N: Número de imagens no blob (1 neste caso).
# - C: Número de canais (1, para probabilidade de texto).
# - H e W: Altura e largura reduzidas da imagem processada pela rede.
scores


array([[[[1.47752374e-07, 5.33552679e-07, 6.18313038e-07, ...,
          2.38874441e-06, 2.14221203e-07, 1.97413760e-06],
         [2.21553137e-07, 8.22969994e-08, 1.52692380e-07, ...,
          3.80215579e-06, 2.49685854e-07, 2.09746190e-06],
         [2.41076435e-07, 2.56505132e-07, 6.74969783e-07, ...,
          4.48429728e-06, 6.08908124e-08, 2.51588767e-06],
         ...,
         [5.19031346e-07, 1.34708145e-07, 3.80787064e-08, ...,
          1.79544452e-06, 7.75353328e-07, 5.47008494e-06],
         [2.85663532e-07, 8.55804032e-08, 1.12898597e-07, ...,
          1.13547708e-06, 9.01572434e-07, 5.07820823e-06],
         [7.21993842e-08, 8.43333137e-09, 5.46042234e-09, ...,
          1.91421819e-07, 3.16591752e-07, 1.02402886e-07]]]],
      dtype=float32)

In [ ]:
# Exibe o conteúdo da variável `geometry`, que contém os dados geométricos.
# `geometry` é uma saída da rede neural que fornece informações sobre a geometria das regiões de texto detectadas.
# Geralmente, é um tensor de 4 dimensões no formato (N, 5, H, W), onde:
# - N: Número de imagens no blob (1 neste caso).
# - 5: Representa os 5 parâmetros geométricos para cada região detectada:
#       (distância do topo, direita, inferior, esquerda e o ângulo de rotação).
# - H e W: Altura e largura reduzidas da imagem processada pela rede.
geometry


array([[[[ 1.54848892e+02,  1.01938652e+02,  1.08211067e+02, ...,
           1.33965744e+02,  1.27577377e+02,  1.46967911e+02],
         [ 1.19407845e+02,  6.46942062e+01,  7.18581467e+01, ...,
           9.02676239e+01,  8.52865753e+01,  1.03320236e+02],
         [ 1.22548698e+02,  6.90612106e+01,  7.86800232e+01, ...,
           1.06370026e+02,  1.01670258e+02,  1.14973625e+02],
         ...,
         [ 1.16457443e+02,  4.99968452e+01,  3.75178909e+01, ...,
           8.21259766e+01,  8.12639771e+01,  1.21695587e+02],
         [ 1.27623566e+02,  5.88201637e+01,  4.84035950e+01, ...,
           8.31772385e+01,  8.09339981e+01,  1.11183853e+02],
         [ 1.26661636e+02,  7.53631363e+01,  6.93250504e+01, ...,
           1.27025375e+02,  1.18697914e+02,  1.47223694e+02]],

        [[ 1.09559792e+02,  9.52129364e+01,  9.08699570e+01, ...,
           1.00495560e+02,  9.43882065e+01,  1.12618568e+02],
         [ 8.27181702e+01,  5.77867126e+01,  5.62353210e+01, ...,
           6.53349457e

In [ ]:
# Exibe o valor do primeiro parâmetro geométrico de uma célula específica na saída `geometry`.
# Aqui:
# - `[0, 0, 0]`: Refere-se à imagem 0 (única no blob), ao primeiro parâmetro geométrico
#   (distância da borda superior), e à célula no índice 0 na dimensão horizontal e vertical da saída.
geometry[0, 0, 0]


array([154.84889 , 101.93865 , 108.21107 , 110.35274 , 111.579956,
       113.072136, 114.39854 , 116.1475  , 115.30822 , 115.90365 ,
       117.415665, 118.78156 , 116.545105, 116.34599 , 121.44332 ,
       129.47726 , 137.6081  , 143.0237  , 146.83202 , 149.02911 ,
       149.08717 , 148.28294 , 147.13005 , 145.16948 , 143.0738  ,
       140.95238 , 138.77005 , 137.29416 , 134.19766 , 131.39839 ,
       129.34814 , 127.54952 , 126.742744, 125.23263 , 120.42206 ,
       117.427246, 116.22169 , 116.099655, 116.90419 , 118.21091 ,
       119.18678 , 123.20284 , 125.41776 , 126.38552 , 126.154854,
       126.193016, 126.85884 , 126.92803 , 128.2181  , 128.52025 ,
       128.97598 , 129.4047  , 130.30916 , 130.73273 , 130.80652 ,
       130.30147 , 129.32219 , 127.84744 , 126.37807 , 125.062874,
       124.11794 , 123.375946, 122.265175, 120.8582  , 120.00341 ,
       119.063995, 117.834236, 116.99429 , 115.744   , 117.560776,
       119.882645, 121.395935, 125.46858 , 121.87925 , 120.994

In [ ]:
# Exibe as dimensões do tensor `scores`, que contém o mapa de probabilidade de texto.
# A função `.shape` retorna a estrutura do array em forma de tupla no formato (N, C, H, W), onde:
# - N: Número de imagens no blob (normalmente 1 neste caso).
# - C: Número de canais (geralmente 1, indicando que é um mapa de probabilidade).
# - H: Altura reduzida da saída da rede em pixels.
# - W: Largura reduzida da saída da rede em pixels.
scores.shape


(1, 1, 80, 80)

In [ ]:
# Obtém as dimensões da grade reduzida de saída para a altura e largura do tensor `scores`.
# O operador `[2:4]` seleciona as dimensões correspondentes à altura (H) e largura (W) reduzidas.
scores.shape[2:4]

(80, 80)

In [ ]:
# Obtém as dimensões da grade reduzida de saída da rede neural, correspondente ao mapa de confiança.
# `scores.shape[2:4]` retorna uma tupla com (altura, largura) da grade reduzida.
# Essas dimensões representam o número de células na grade da saída da rede.
linhas, colunas = scores.shape[2:4]


In [ ]:
# Imprime as dimensões da grade reduzida de saída da rede neural.
# `linhas` representa o número de células na direção vertical.
# `colunas` representa o número de células na direção horizontal.
print(linhas, colunas)


80 80


In [ ]:
# Inicializa uma lista vazia para armazenar as caixas delimitadoras (bounding boxes) das regiões de texto detectadas.
# Cada elemento dessa lista será uma caixa delimitadora representada por coordenadas (x, y, largura, altura).
caixas = []

# Inicializa uma lista vazia para armazenar os valores de confiança (scores) associados a cada caixa detectada.
# Esses valores indicam a probabilidade de a região corresponder a texto.
confiancas = []


# Decodificando os valores

- Explicação sobre os [valores de geometria](https://stackoverflow.com/questions/55583306/decoding-geometry-output-of-east-text-detection)

In [ ]:
# Define uma função para extrair os dados geométricos de uma linha específica do tensor `geometry`.
# Essa função é usada para obter as informações necessárias para reconstruir as caixas delimitadoras.
def dados_geometricos(geometry, y):
    # Extrai os deslocamentos das bordas superior, direita, inferior e esquerda para a linha `y`.
    # Esses dados são armazenados nas diferentes camadas do tensor `geometry`:
    # - geometry[0, 0, y]: Deslocamento da borda superior.
    # - geometry[0, 1, y]: Deslocamento da borda direita.
    # - geometry[0, 2, y]: Deslocamento da borda inferior.
    # - geometry[0, 3, y]: Deslocamento da borda esquerda.
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]

    # Extrai o ângulo de rotação associado à célula da linha `y`.
    # O ângulo é armazenado na camada geometry[0, 4, y].
    data_angulos = geometry[0, 4, y]

    # Retorna os dados geométricos para a linha `y`:
    # - `data_angulos`: Ângulo de rotação.
    # - `xData0`, `xData1`, `xData2`, `xData3`: Deslocamentos das bordas.
    return data_angulos, xData0, xData1, xData2, xData3


In [ ]:
# Define uma função para calcular as coordenadas das caixas delimitadoras (bounding boxes) a partir dos dados geométricos.
def calculos_geometria(data_angulos, xData0, xData1, xData2, xData3):
    # Calcula os deslocamentos na escala original da imagem para o ponto de referência (célula da grade).
    # Multiplica os índices `x` e `y` da célula por 4, já que o mapa de saída é reduzido em relação à imagem original.
    (offsetX, offsetY) = (x * 4.0, y * 4.0)

    # Obtém o ângulo de rotação associado à célula.
    angulo = data_angulos[x]

    # Calcula os valores de cosseno e seno do ângulo, necessários para ajustar a rotação da caixa.
    cos = np.cos(angulo)
    sin = np.sin(angulo)

    # Calcula a altura (`h`) e a largura (`w`) da caixa delimitadora usando os deslocamentos geométricos.
    h = xData0[x] + xData2[x]  # Soma dos deslocamentos superior e inferior.
    w = xData1[x] + xData3[x]  # Soma dos deslocamentos direito e esquerdo.

    # Calcula as coordenadas do canto inferior direito da caixa delimitadora (fimX, fimY).
    fimX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
    fimY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))

    # Calcula as coordenadas do canto superior esquerdo da caixa delimitadora (inicioX, inicioY).
    inicioX = int(fimX - w)
    inicioY = int(fimY - h)

    # Retorna as coordenadas completas da caixa delimitadora:
    # - (inicioX, inicioY): Canto superior esquerdo.
    # - (fimX, fimY): Canto inferior direito.
    return inicioX, inicioY, fimX, fimY


In [ ]:
# `linhas` e `colunas` representam as dimensões da grade reduzida de saída da rede neural.
# Essas variáveis já foram obtidas anteriormente a partir de `scores.shape[2:4]`:
# - `linhas`: Número de células na direção vertical (altura da grade reduzida).
# - `colunas`: Número de células na direção horizontal (largura da grade reduzida).
linhas, colunas


(80, 80)

In [ ]:
# Acessa o valor do mapa de probabilidade de texto para a célula específica na linha 0 e coluna 0.
# `scores` é o mapa de confiança retornado pela rede neural, representando a probabilidade de cada região conter texto.
# Aqui:
# - `0`: Refere-se à primeira (e única) imagem no blob de entrada.
# - `0`: Refere-se ao único canal (probabilidade de texto).
# - `0`: Índice da linha na grade reduzida.
# - O valor retornado é a confiança (score) da célula localizada na linha 0 e coluna 0.
scores[0, 0, 0]


array([1.47752374e-07, 5.33552679e-07, 6.18313038e-07, 5.44988495e-07,
       6.83899884e-07, 6.35390847e-07, 2.87392936e-07, 1.17966316e-07,
       1.41452034e-07, 1.98707312e-07, 1.92506320e-07, 1.78495881e-07,
       3.21175833e-07, 1.10004373e-06, 1.85389433e-06, 7.20500395e-07,
       2.26942120e-07, 1.93052713e-07, 2.77832100e-07, 4.64349341e-07,
       5.43345493e-07, 6.31639011e-07, 6.57134535e-07, 6.78435356e-07,
       6.93763980e-07, 5.52085112e-07, 4.24791835e-07, 3.10167337e-07,
       4.78031438e-07, 4.52927878e-07, 5.14709882e-07, 5.01000329e-07,
       3.54018198e-07, 4.92441586e-07, 2.26114366e-06, 3.20661206e-06,
       2.21842652e-06, 1.91733261e-06, 1.95528651e-06, 2.52953100e-06,
       3.17436297e-06, 2.37437212e-06, 3.73728085e-07, 5.99255259e-08,
       7.66571731e-08, 1.31929426e-07, 1.90021836e-07, 3.26953739e-07,
       1.99290312e-07, 1.49426484e-07, 2.11751313e-07, 2.15443109e-07,
       2.36163473e-07, 3.09929874e-07, 3.78356759e-07, 4.72775469e-07,
      

In [ ]:
# Itera sobre cada linha na grade reduzida da saída da rede neural.
for y in range(0, linhas):
    # (Opcional) Imprime o índice da linha atual para depuração.
    # print(y)

    # Obtém o mapa de probabilidade para a linha atual.
    # `data_scores` contém os valores de confiança (scores) para todas as células na linha `y`.
    data_scores = scores[0, 0, y]

    # Extrai os dados geométricos e ângulo de rotação para a linha atual.
    # Esses dados são usados para calcular as caixas delimitadoras.
    data_angulos, xData0, xData1, xData2, xData3 = dados_geometricos(geometry, y)

    # Itera sobre cada célula na linha atual.
    for x in range(0, colunas):
        # Verifica se o score da célula é menor que o limite de confiança.
        # Se for, pula para a próxima célula (célula descartada por baixa probabilidade).
        if data_scores[x] < min_confianca:
            continue

        # Calcula as coordenadas da caixa delimitadora para a célula atual.
        # As coordenadas são calculadas usando os dados geométricos e a posição da célula.
        inicioX, inicioY, fimX, fimY = calculos_geometria(data_angulos, xData0, xData1, xData2, xData3)

        # Adiciona o valor de confiança da célula à lista `confiancas`.
        # Isso será usado para avaliar a relevância das caixas detectadas.
        confiancas.append(data_scores[x])

        # Adiciona as coordenadas da caixa delimitadora à lista `caixas`.
        # Cada caixa é representada como uma tupla: (inicioX, inicioY, fimX, fimY).
        caixas.append((inicioX, inicioY, fimX, fimY))

In [ ]:
# A lista `confiancas` armazena os valores de probabilidade (scores) para cada caixa delimitadora detectada.
# Esses valores indicam a confiança da rede neural de que a região contém texto.
# Apenas as células que têm probabilidade maior ou igual a `min_confianca` são adicionadas à lista.
confiancas

[0.9995617,
 0.99982554,
 0.9997718,
 0.9994678,
 0.9662514,
 0.96421045,
 0.9993309,
 0.99968016,
 0.99951243]

In [ ]:
# A lista `caixas` armazena as coordenadas das caixas delimitadoras (bounding boxes) detectadas.
# Cada caixa corresponde a uma região de texto identificada pela rede neural.
# As caixas são representadas como tuplas: (inicioX, inicioY, fimX, fimY), onde:
# - `inicioX`, `inicioY`: Coordenadas do canto superior esquerdo da caixa.
# - `fimX`, `fimY`: Coordenadas do canto inferior direito da caixa.
caixas


[(156, 208, 190, 220),
 (157, 208, 187, 220),
 (157, 208, 186, 220),
 (158, 207, 186, 220),
 (156, 207, 187, 221),
 (157, 207, 188, 220),
 (158, 208, 187, 221),
 (158, 208, 187, 221),
 (157, 208, 188, 221)]

In [ ]:
# Aplica a supressão não máxima (Non-Max Suppression - NMS) para eliminar caixas delimitadoras sobrepostas.
# `non_max_suppression` remove caixas redundantes mantendo apenas a mais confiável em cada região.
# Isso ajuda a reduzir ruído e melhorar a precisão das detecções.

# `np.array(caixas)`: Converte a lista de caixas delimitadoras em um array NumPy, necessário para a função NMS.
# `probs=confiancas`: Passa os valores de confiança associados a cada caixa para que a NMS priorize as mais confiáveis.
deteccoes = non_max_suppression(np.array(caixas), probs=confiancas)


In [ ]:
# A variável `deteccoes` armazena as caixas delimitadoras finais após a aplicação de supressão não máxima (NMS).
# Cada elemento em `deteccoes` é uma caixa representada como uma tupla de coordenadas: (inicioX, inicioY, fimX, fimY).
# Essas caixas são as mais confiáveis e não possuem sobreposições redundantes com outras caixas.
deteccoes


array([[157, 208, 187, 220]])

In [ ]:
# `proporcao_H` e `proporcao_W` representam as proporções de redimensionamento da imagem original para a entrada da rede.
# Eles são usados para mapear as coordenadas das caixas delimitadoras da escala reduzida de volta para a escala original.
# - `proporcao_H`: Proporção entre a altura da imagem original e a altura da imagem redimensionada (H).
# - `proporcao_W`: Proporção entre a largura da imagem original e a largura da imagem redimensionada (W).
proporcao_H, proporcao_W


(1.5, 2.34375)

In [ ]:
# Cria uma cópia da imagem original para preservar a imagem sem alterações.
# A cópia será usada para desenhar as caixas delimitadoras sem afetar a imagem original.
copia = original.copy()

# Itera sobre as caixas delimitadoras detectadas na lista `deteccoes`.
for (inicioX, inicioY, fimX, fimY) in deteccoes:
    # Ajusta as coordenadas das caixas de volta para a escala original da imagem
    inicioX = int(inicioX * proporcao_W)  # Multiplica pela proporção da largura
    inicioY = int(inicioY * proporcao_H)  # Multiplica pela proporção da altura
    fimX = int(fimX * proporcao_W)        # Multiplica pela proporção da largura
    fimY = int(fimY * proporcao_H)        # Multiplica pela proporção da altura

    # Define a região de interesse (ROI) da imagem, que corresponde à caixa delimitadora.
    # A ROI é a parte da imagem que foi identificada como texto.
    roi = copia[inicioY:fimY, inicioX:fimX]

    # Desenha um retângulo verde na imagem original para marcar a área detectada como texto.
    # A função `cv2.rectangle` desenha o retângulo nas coordenadas ajustadas.
    cv2.rectangle(original, (inicioX, inicioY), (fimX, fimY), (0, 255, 0), 2)

# Exibe a imagem com as caixas delimitadoras desenhadas.
cv2_imshow(original)


NameError: name 'original' is not defined

In [ ]:
# Exibe a região de interesse (ROI) extraída da imagem.
# A ROI corresponde à área da imagem que foi identificada como texto, ou seja,
# a parte da imagem dentro das coordenadas da caixa delimitadora (inicioX, inicioY, fimX, fimY).
cv2_imshow(roi)


In [ ]:
# Redimensiona a região de interesse (ROI) para aumentar sua resolução.
# O redimensionamento é feito usando o fator de escala (fx, fy) de 1.5, ou seja, aumentando 50% da largura e altura.
# A interpolação `cv2.INTER_CUBIC` é usada para obter uma melhor qualidade na ampliação da imagem.
# Isso é útil quando a ROI precisa ser ampliada para melhor visualização ou análise.
roi = cv2.resize(roi, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)


In [ ]:
cv2_imshow(roi)

# Reconhecimento do texto

In [ ]:
# Instala o Tesseract OCR, um mecanismo de reconhecimento óptico de caracteres.
# O Tesseract é necessário para realizar o OCR em imagens e extrair o texto delas.
!sudo apt install tesseract-ocr

# Instala o pacote `pytesseract`, que é a interface Python para o Tesseract OCR.
# O `pytesseract` permite que você use o Tesseract diretamente do Python para realizar OCR.
!pip install pytesseract

# Importa o módulo `pytesseract`, que fornece as funções para usar o Tesseract OCR.
import pytesseract


*Aviso: em setembro de 2021 o link para baixar o **por.traineddata** sofreu uma pequena alteração [ para mais explicações consulte a aula "Resolvendo problema relacionado ao pacote de idiomas" ]*

Portanto, o link final ficou: https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true (segue abaixo o comando completo)

In [ ]:
# Cria um diretório chamado `tessdata`, onde o modelo de idioma para o Tesseract será armazenado.
# Esse diretório é usado para armazenar arquivos de dados treinados, como o modelo de idioma para o português.
!mkdir tessdata

# Baixa o arquivo de dados treinados do idioma português (`por.traineddata`) do repositório oficial do Tesseract no GitHub.
# Esse arquivo contém o modelo necessário para que o Tesseract reconheça texto em português.
# O arquivo é salvo no diretório `tessdata` que foi criado anteriormente.
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true


--2021-10-10 23:01:50--  https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata [following]
--2021-10-10 23:01:50--  https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata [following]
--2021-10-10 23:01:50--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [ ]:
# Exibe a ajuda sobre os modos de segmentação de página (PSM) do Tesseract.
# O Tesseract utiliza diferentes modos de segmentação para lidar com diferentes layouts de texto nas imagens.
# Esse comando lista as opções disponíveis para a segmentação de página, que ajudam o Tesseract a entender
# como organizar e interpretar o texto na imagem.
!tesseract --help-psm


Page segmentation modes:
  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR.
  3    Fully automatic page segmentation, but no OSD. (Default)
  4    Assume a single column of text of variable sizes.
  5    Assume a single uniform block of vertically aligned text.
  6    Assume a single uniform block of text.
  7    Treat the image as a single text line.
  8    Treat the image as a single word.
  9    Treat the image as a single word in a circle.
 10    Treat the image as a single character.
 11    Sparse text. Find as much text as possible in no particular order.
 12    Sparse text with OSD.
 13    Raw line. Treat the image as a single text line,
       bypassing hacks that are Tesseract-specific.


In [ ]:
# Configura as opções para o Tesseract:
# - `--tessdata-dir tessdata`: Especifica o diretório onde os dados treinados do Tesseract estão armazenados.
#   Isso é necessário para informar ao Tesseract onde encontrar os modelos de idioma (por exemplo, `por.traineddata`).
# - `--psm 7`: Define o modo de segmentação de página como `7`, que instrui o Tesseract a tratar a imagem como uma única linha de texto.
# Isso é útil para imagens com texto em uma única linha ou em um formato de linha contínua.
config_tesseract = "--tessdata-dir tessdata --psm 7"


In [ ]:
# Cria uma cópia da imagem original para não modificar a imagem original ao desenhar as caixas delimitadoras.
copia = original.copy()

# Itera sobre as caixas delimitadoras detectadas em `deteccoes`.
# Cada caixa é representada pelas coordenadas (inicioX, inicioY, fimX, fimY).
for (inicioX, inicioY, fimX, fimY) in deteccoes:
    # Ajusta as coordenadas das caixas para a escala original da imagem.
    inicioX = int(inicioX * proporcao_W)
    inicioY = int(inicioY * proporcao_H)
    fimX = int(fimX * proporcao_W)
    fimY = int(fimY * proporcao_H)

    # Extrai a região de interesse (ROI) da imagem, correspondente à caixa delimitadora.
    # A ROI é a área que o Tesseract irá processar para realizar o OCR.
    roi = copia[inicioY:fimY, inicioX:fimX]

    # Usa o Tesseract para realizar OCR na ROI extraída, especificando o idioma ('por' para português) e a configuração de PSM.
    texto = pytesseract.image_to_string(roi, lang='por', config=config_tesseract)
    print(texto)  # Exibe o texto extraído da ROI.

    # Desenha um retângulo verde na imagem original para marcar a caixa delimitadora.
    # A caixa é desenhada nas coordenadas ajustadas.
    cv2.rectangle(original, (inicioX, inicioY), (fimX, fimY), (0, 255, 0), 2)

# Exibe a imagem original com as caixas delimitadoras desenhadas.
cv2_imshow(original)


NameError: name 'original' is not defined

## Expandindo o ROI do texto

In [ ]:
# Define uma margem extra ao redor das caixas delimitadoras para capturar mais contexto de texto.
margem = 3

# Cria uma cópia da imagem original para desenhar as caixas delimitadoras sem alterar a imagem original.
copia = original.copy()

# Itera sobre as caixas delimitadoras detectadas na lista `deteccoes`.
# Cada caixa é representada pelas coordenadas (inicioX, inicioY, fimX, fimY).
for (inicioX, inicioY, fimX, fimY) in deteccoes:
    # Ajusta as coordenadas das caixas para a escala original da imagem.
    inicioX = int(inicioX * proporcao_W)
    inicioY = int(inicioY * proporcao_H)
    fimX = int(fimX * proporcao_W)
    fimY = int(fimY * proporcao_H)

    # Extraí a região de interesse (ROI) da imagem, adicionando uma margem extra ao redor da caixa.
    # A margem é usada para capturar um pouco mais de contexto da imagem em torno do texto detectado.
    roi = copia[inicioY - margem:fimY + margem, inicioX - margem:fimX + margem]

    # Exibe a ROI extraída, com a margem, para ver o conteúdo que será processado.
    cv2_imshow(roi)

    # Usa o Tesseract para realizar OCR na ROI extraída.
    texto = pytesseract.image_to_string(roi, lang='por', config=config_tesseract)
    print(texto)  # Exibe o texto extraído da ROI.

    # Desenha um retângulo verde na imagem, incluindo a margem, para marcar a área da caixa delimitadora.
    cv2.rectangle(copia, (inicioX - margem, inicioY - margem), (fimX + margem, fimY + margem), (0, 255, 0), 2)

# Exibe a imagem original com as caixas delimitadoras (e margens) desenhadas.
cv2_imshow(copia)

NameError: name 'original' is not defined